In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.layers import LSTM, Dense, Bidirectional, Dropout


In [7]:

# Read the data from the CSV file into a DataFrame, specifying the encoding and column names
df = pd.read_csv('data.csv', header=None, names=['Draw', 'Date', 'Numbers'], encoding='utf-8')

# Split the Numbers column into separate columns
df[['Number1', 'Number2', 'Number3']] = df['Numbers'].str.split('-', expand=True)

# Drop the original Numbers column
df.drop('Numbers', axis=1, inplace=True)


df.drop('Date', axis=1, inplace=True)
df.drop('Draw', axis=1, inplace=True)

df.head()


,Number1,Number2,Number3
0,6,9,2
1,5,4,8
2,1,6,9
3,5,9,0
4,5,0,4


In [8]:
print(df.shape)
df.info()

(19980, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19980 entries, 0 to 19979
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Number1  19980 non-null  object
 1   Number2  19980 non-null  object
 2   Number3  19980 non-null  object
dtypes: object(3)
memory usage: 468.4+ KB


In [9]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()

,0,1,2
0,0.512064,1.556031,-0.866826
1,0.163963,-0.180250,1.228743
2,-1.228442,0.514262,1.578005
3,0.163963,1.556031,-1.565349
4,0.163963,-1.569275,-0.168303


In [5]:
number_of_rows = df.values.shape[0]
number_of_rows

19980

In [6]:
window_length = 7
window_length
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

3

In [7]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [8]:
print(X.shape)
print(y.shape)

(19973, 7, 3)
(19973, 3)


In [9]:
from keras import models
model = models.Sequential()

In [10]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [11]:
model.add(Dropout(0.2))

In [12]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [13]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))
model.add(Dropout(0.2))

In [14]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [15]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))

In [16]:
model.add(Dense(59))

In [17]:
model.add(Dense(number_of_features))

In [18]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [19]:
model.fit(x=X, y=y, batch_size=100, epochs=300, verbose=2)

Epoch 1/300
200/200 - 163s - loss: 1.0004 - accuracy: 0.3387 - 163s/epoch - 815ms/step
Epoch 2/300
200/200 - 138s - loss: 1.0001 - accuracy: 0.3308 - 138s/epoch - 690ms/step
Epoch 3/300
200/200 - 144s - loss: 1.0000 - accuracy: 0.3449 - 144s/epoch - 722ms/step
Epoch 4/300
200/200 - 144s - loss: 1.0000 - accuracy: 0.3336 - 144s/epoch - 720ms/step
Epoch 5/300
200/200 - 150s - loss: 1.0000 - accuracy: 0.3346 - 150s/epoch - 751ms/step
Epoch 6/300
200/200 - 152s - loss: 1.0000 - accuracy: 0.3371 - 152s/epoch - 758ms/step
Epoch 7/300
200/200 - 153s - loss: 0.9999 - accuracy: 0.3417 - 153s/epoch - 765ms/step
Epoch 8/300
200/200 - 151s - loss: 1.0000 - accuracy: 0.3318 - 151s/epoch - 757ms/step
Epoch 9/300
200/200 - 151s - loss: 0.9998 - accuracy: 0.3417 - 151s/epoch - 754ms/step
Epoch 10/300
200/200 - 156s - loss: 0.9999 - accuracy: 0.3259 - 156s/epoch - 778ms/step
Epoch 11/300
200/200 - 158s - loss: 0.9998 - accuracy: 0.3345 - 158s/epoch - 790ms/step
Epoch 12/300
200/200 - 157s - loss: 0.999

In [83]:
import pickle

#load a saved model
with open('model', 'rb') as f:
   model =  pickle.load(f)


In [99]:
# Read the data from the CSV file into a DataFrame, specifying the encoding and column names
df_input = pd.read_csv('input.csv', header=None, names=['Draw', 'Date', 'Numbers'], encoding='utf-8')

# Replace non-breaking spaces with regular spaces in the Numbers column
df_input['Numbers'] = df_input['Numbers'].str.replace('\xa0', ' ')

# Split the Numbers column into separate columns
df_input[['Number1', 'Number2', 'Number3']] = df_input['Numbers'].str.split('-', expand=True)

# Drop the original Numbers, Date and Draw columns
df_input.drop(['Numbers', 'Date', 'Draw'], axis=1, inplace=True)

# Convert DataFrame to NumPy array and convert to int data type
np_array = df_input.values.astype(np.int)

print(np_array)

[[1 2 1]
 [2 7 1]
 [9 1 2]
 [0 4 9]
 [5 2 2]
 [7 7 3]
 [3 8 8]]


C:\Users\mwasj\AppData\Local\Temp\ipykernel_1056\3106214009.py:14: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np_array = df_input.values.astype(np.int)


In [100]:

scaled_to_predict = scaler.transform(np_array)
scaled_to_predict

array([[-1.22844193, -0.87476222, -1.21608757],
       [-0.8803407 ,  0.86151851, -1.21608757],
       [ 1.55636792, -1.22201836, -0.86682599],
       [-1.57654316, -0.18024992,  1.57800507],
       [ 0.163963  , -0.87476222, -0.86682599],
       [ 0.86016546,  0.86151851, -0.51756441],
       [-0.53223947,  1.20877466,  1.22874349]])

In [101]:
y_pred = model.predict(np.array([scaled_to_predict]))

scaler.inverse_transform(y_pred).astype(int)[0]

1/1 [==============================] - 0s 122ms/step


array([5, 7, 3])